# 抽出型文書要約

## 手続き

1. 前準備
1. クリーニング(文区切り，絵文字/記号などの削除)
1. 文章の正規化(全角半角変換，数字の削除)
1. 文章の単語分割(形態素解析)
1. 単語の正規化(同じ意味を持つ単語の統一)
1. ストップワードの除去(名詞/形容詞/副詞/動詞のみの利用，辞書による除去)
1. 文書のベクトル表現(TF-IDF)
1. 要約モデルの適用(LexRank)
1. FlaskでAPIに

## 実装

### 1. 前準備

In [133]:
import numpy as np
import pandas as pd

# 文区切り
import functools
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

# クリーニング
import re
from experiment.modules import data_cleaning as dc

# MeCab(形態素解析)
import MeCab
tagger = MeCab.Tagger('-r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 単語の正規化
import emoji
import mojimoji

# ストップワード除去
# 名詞/形容詞/副詞/動詞のみを抽出できるライブラリのインポート

# 抽出型要約モデル(LexRank)
import sumy

### 2. クリーニング

In [134]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)([\r\n]+)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)

text1 = """
魚にケチョンケチョンにやられて今にもロッド離しそう・・・。そんな時は？
握力無くなってロッドを離しそうな時は？
パワーグリップって使えるの？
最近筋トレで使用していたものなんですけど、
パワーグリップ って知ってます？？
ウェイトトレーニングで使う物なんですけど、これ、何に使うのかと言うと、高負荷の重量を持つ時とかって、握力が先に尽きてウェイトが持てなくなる時あるんですよね。
その時に使う物なんですけど、よくデッドリフトとかチンニングなんかで使っているのかな？
私は背筋鍛えるのに使用しているのですが、これあると簡単に１００キロとかそれ以上のバーベルを掴んでしばらくの間持ち上げられるんですね。
あれ？これ釣りに使えるんじゃない？
って思ったんですよ。
こんな感じに。
あーいつまででもロッド持っていられそう！！！
100キロかかっても、フルドラグで持ってられそう！！！
と、思ったんですけど、ちょっと微妙かなｗ
普通に実用できる範囲なんですけど、なんかのトラブルでラインに引っ掛かったりして腕持ってかれたりする可能性あるのでここではオススメはしないですな(^^;
※グリップ握ってラインまで届く代物ではないですけど。
そもそも、大物釣りってロッドのグリップをチョコチョコ持ち替えたりするんですよね。
遠くにいる時と足元いる時ではファイトの方法少し違うし。
うーーんって感じｗでも、ちょっと改造したらいいアイテムになりそうだな。
メーカーさん、良い案ないですか？
売れるかもしれないですよｗ
"""

In [135]:
sentences = list(segmenter(text1))
pd.DataFrame(sentences)

,0
0,魚にケチョンケチョンにやられて今にもロッド離しそう・・・。
1,そんな時は?
2,握力無くなってロッドを離しそうな時は?
3,パワーグリップって使えるの?
4,最近筋トレで使用していたものなんですけど、
5,パワーグリップって知ってます?
6,?
7,ウェイトトレーニングで使う物なんですけど、これ、何に使うのかと言うと、高負荷の重量を持つ時と...
8,その時に使う物なんですけど、よくデッドリフトとかチンニングなんかで使っているのかな?
9,私は背筋鍛えるのに使用しているのですが、これあると簡単に100キロとかそれ以上のバーベルを掴...


### 3. 文章の正規化

In [136]:
sentences_dc = dc.data_cleaning(sentences)
pd.DataFrame(sentences_dc)

,0
0,魚にケチョンケチョンにやられて今にもロッド離しそう・・・。
1,そんな時は
2,握力無くなってロッドを離しそうな時は
3,パワーグリップって使えるの
4,最近筋トレで使用していたものなんですけど、
5,パワーグリップって知ってます
6,
7,ウェイトトレーニングで使う物なんですけど、これ、何に使うのかと言うと、高負荷の重量を持つ時と...
8,その時に使う物なんですけど、よくデッドリフトとかチンニングなんかで使っているのかな
9,私は背筋鍛えるのに使用しているのですが、これあると簡単に0キロとかそれ以上のバーベルを掴んで...


In [ ]:
text = "".join(lemmatize_term(e) for e in text.split())

In [ ]:
def lemmatize_term(term, pos=None):
    if pos is None:
        synsets = wordnet.synsets(term, lang='jpn')
        if not synsets:
            return term
        pos = synsets[0].pos()
        if pos == wordnet.ADJ_SAT:
            pos = wordnet.ADJ
    return nltk.WordNetLemmatizer().lemmatize(term, pos=pos)